# Exportar os json com os dados mais importantes/relevantes
- business: business_id; name; city; latitude; longitude; stars; categories
- review: review_id; user_id; business_id; stars; date; # texto não será utilizado provavelmente, se for so mudar
- user: nada
- checkin: nada
- tip: user_id; business_id; date # não será utilizado pois não tem stars

categories não será transformado em feather pois o formato json é valido.

em https://www.yelp.com/dataset/documentation/main a yelp diz que o categories é uma array de string, porém isso é falso, no json que eles dispõem o categories é uma string somente. 2019-08-11 02:16:40

Existe business com categorias nulas.

In [54]:
import sys
sys.path.append('../')
import json
import pandas as pd
import networkx as nx
import algorithms.areamanager as areamanager
import numpy as np
import pyarrow
cities_name=['madison']#,'lasvegas','phoenix','charllote']
df_categories=pd.read_json("../data/categories.json")

# dicionário alias title 2 way
dict_alias_title=dict()
for index, row in df_categories.iterrows():
    dict_alias_title[row['alias']]=row['title']
    dict_alias_title[row['title']]=row['alias']


category_tree= nx.DiGraph()
for index, row in df_categories.iterrows():
    if not row['parents']:
        category_tree.add_edge(row['alias'],'root') # root node if no parents
    else:
        for parent_label in row['parents']:
            category_tree.add_edge(row['alias'],parent_label)

undirected_category_tree=category_tree.to_undirected()
# dict alias depth
dict_alias_depth=dict()
for index, row in df_categories.iterrows():
    dict_alias_depth[row['alias']]=nx.shortest_path_length(category_tree,row['alias'],'root')
df_categories=None


def string_to_array(string):
    if string == '':
        return list()
    return eval(string)
def category_filter(categories):
    tmp_cat_list=list()
    for category in categories:
        try:
            if dict_alias_depth[dict_alias_title[category]] <= 2:
                tmp_cat_list.append(category)
        except:
            pass
    tmp_cat_list=geocat.get_most_detailed_categories(tmp_cat_list)
#             print("Error: Category \""+category+"\" without alias name.")
    return tmp_cat_list

In [2]:
fbusiness=open("../data/business.json")
all_data = list()
for i, line in enumerate(fbusiness):  
    # json to dict
    business_json = json.loads(line)
    # add to the data collection
    if business_json['categories'] != None:
        all_data.append([business_json['business_id'],
                         business_json['name'],
                         business_json['city'],
                         business_json['latitude'],
                         business_json['longitude'],
                         business_json['stars'],
                         business_json['categories'].split(', ')])
    else:
        all_data.append([business_json['business_id'],
                 business_json['name'],
                 business_json['city'],
                 business_json['latitude'],
                 business_json['longitude'],
                 business_json['stars'],
                 business_json['categories']])
# create the DataFrame
df = pd.DataFrame(all_data, columns=['business_id', 'name', 'city','latitude', 'longitude', 'stars', 'categories'])

df.head()

,business_id,name,city,latitude,longitude,stars,categories
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,Phoenix,33.522143,-112.018481,3.0,"[Golf, Active Life]"
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,Mississauga,43.605499,-79.652289,2.5,"[Specialty Food, Restaurants, Dim Sum, Importe..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,Charlotte,35.092564,-80.859132,4.0,"[Sushi Bars, Restaurants, Japanese]"
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,Goodyear,33.455613,-112.395596,5.0,"[Insurance, Financial Services]"
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,Charlotte,35.190012,-80.887223,4.0,"[Plumbing, Shopping, Local Services, Home Serv..."


In [3]:
df.to_csv('../data/business.csv',index=False)
#df.to_pickle('../data/business.pickle')
#df.to_feather('../data/business.feather')
#df.to_parquet('../data/business.parquet')
df=None

In [4]:
freview=open("../data/review.json")
all_data = list()
df = pd.DataFrame(columns=['review_id', 'user_id', 'business_id', 'date'])
df.to_csv('../data/review.csv',index=False)

for i, line in enumerate(freview):  
    # json to dict
    business_json = json.loads(line)
    # add to the data collection
    all_data.append([business_json['review_id'],
                     business_json['user_id'],
                     business_json['business_id'],
                     business_json['date']])
    if i % 1000000==0:
        df = pd.DataFrame(all_data, columns=['review_id', 'user_id', 'business_id', 'date'])
        df.to_csv('../data/review.csv',header=False,index=False,mode='a')
        df=None
        all_data = list()
# create the DataFrame

df = pd.DataFrame(all_data, columns=['review_id', 'user_id', 'business_id', 'date'])
df.to_csv('../data/review.csv',header=False,index=False,mode='a')
df=None
all_data = list()


In [5]:

#df.to_csv('../data/review.csv')
#df.to_pickle('../data/review.pickle')
#df.to_feather('../data/review.feather')
#df.to_parquet('../data/review.parquet')

In [6]:
ftip=open("../data/tip.json")
all_data = list()
df = pd.DataFrame(columns=['user_id', 'business_id', 'date'])
df.to_csv('../data/tip.csv',index=False)


for i, line in enumerate(ftip):  
    # json to dict
    tip_json = json.loads(line)
    # add to the data collection
    all_data.append([tip_json['user_id'],
                     tip_json['business_id'],
                     tip_json['date']])
    if i % 100000==0:
        df = pd.DataFrame(all_data, columns=['user_id', 'business_id','date'])
        df.to_csv("../data/tip.csv",header=False,index=False,mode='a')
        df=None
        all_data = list()
# create the DataFrame

df = pd.DataFrame(all_data, columns=['user_id', 'business_id', 'date'])
df.to_csv("../data/tip.csv",header=False,index=False, mode='a')
df=None
all_data = list()


# Export checkin and poi

In [47]:
df_business=pd.read_csv("../data/business.csv",usecols=['business_id','latitude','longitude','categories'],converters={'categories':string_to_array})
df_business['categories']=df_business.categories.apply(category_filter)
df_business.head()

,business_id,latitude,longitude,categories
0,1SWheh84yJXfytovILXOAQ,33.522143,-112.018481,"[Golf, Active Life]"
1,QXAEGFB4oINsVuTFxEYKFQ,43.605499,-79.652289,"[Specialty Food, Restaurants, Imported Food, F..."
2,gnKjwL_1w79qoiV3IC_xQQ,35.092564,-80.859132,"[Sushi Bars, Restaurants, Japanese]"
3,xvX2CttrVhyG2z1dFg_0xw,33.455613,-112.395596,"[Insurance, Financial Services]"
4,HhyxOkGAM07SRYtlQ4wMFQ,35.190012,-80.887223,"[Plumbing, Shopping, Local Services, Home Serv..."


In [48]:
df_business.to_csv("../data/poi.csv",index=False)

In [49]:
# read review and merge with business
df_review=pd.read_csv("../data/review.csv",usecols=['business_id','user_id','date'])
df_tip=pd.read_csv("../data/tip.csv",usecols=['business_id','user_id','date'])
df_checkin=pd.concat([df_review, df_tip], ignore_index=True)
df_review=None
df_tip=None
df_checkin=pd.merge(df_checkin,df_business,on='business_id')
df_checkin['date']=pd.to_datetime(df_checkin['date'])
df_checkin.head()

,user_id,business_id,date,latitude,longitude,categories
0,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,2013-05-07 04:34:36,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Health & Medi..."
1,RBXSJA372ilErzNwz0jXvQ,ujmEBvifdJM6h6RLv4wQIg,2017-02-09 06:45:18,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Health & Medi..."
2,x3brMMbJrAW9PwW5A6YL5w,ujmEBvifdJM6h6RLv4wQIg,2018-08-14 05:25:33,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Health & Medi..."
3,Skzdl0sWhW88525a1vr59g,ujmEBvifdJM6h6RLv4wQIg,2018-05-08 19:27:59,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Health & Medi..."
4,3Y25VDfnQVcuc33T-U3Z6A,ujmEBvifdJM6h6RLv4wQIg,2014-10-11 06:57:18,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Health & Medi..."


In [50]:
df_checkin.to_csv("../data/checkin.csv",index=False)

# Export cities dataset

In [4]:
def export_city(area_case):

    city_area=areamanager.delimiter_area(area_case)
    # read business and do some filterings
    df_business=pd.read_csv("../data/business.csv",usecols=['business_id','latitude','longitude','categories'],converters={'categories':string_to_array})
    df_business=areamanager.pois_in_area(city_area,df_business)
    df_business['categories']=df_business.categories.apply(category_filter)
    # read review and merge with business
    df_review=pd.read_csv("../data/review.csv",usecols=['business_id','user_id','date'])
    df_tip=pd.read_csv("../data/tip.csv",usecols=['business_id','user_id','date'])
    df_checkin=pd.concat([df_review, df_tip], ignore_index=True)
    df_review=None
    df_tip=None
    df_checkin=pd.merge(df_checkin,df_business,on='business_id')
    df_checkin['date']=pd.to_datetime(df_checkin['date'])
    df_checkin.head()


    df_diff_users_visited=df_checkin[['user_id','business_id']].drop_duplicates().reset_index(drop=True).\
    groupby('business_id').count().reset_index().rename(columns={"user_id":"diffusersvisited"})

    df_diff_users_visited=df_diff_users_visited[df_diff_users_visited['diffusersvisited']>=5]
    del df_diff_users_visited['diffusersvisited']
    df_checkin=pd.merge(df_checkin,df_diff_users_visited,on='business_id',how='inner')
    df_checkin.count()


    df_checkin['Count']=df_checkin.groupby(['user_id'])['user_id'].transform('count')
    df_checkin=df_checkin[df_checkin['Count']>=20]
    del df_checkin['Count']
    df_checkin.count()

    areamanager.poi_set_subarea(city_area,df_checkin,0.5)

    return df_checkin


In [5]:
export_city("lasvegas").to_csv("../data/checkin/lasvegas.csv",index=False)

Area selected: Las Vegas


In [6]:
export_city("phoenix").to_csv("../data/checkin/phoenix.csv",index=False)

Area selected: Phoenix


In [7]:
export_city("charlotte").to_csv("../data/checkin/charlotte.csv",index=False)

Area selected: Charlotte


In [8]:
export_city("madison").to_csv("../data/checkin/madison.csv",index=False)

Area selected: Madison


In [9]:
def city_pois(area_case):
    
    city_area=areamanager.delimiter_area(area_case)

    # read business and do some filterings
    df_business=pd.read_csv("../data/business.csv",usecols=['business_id','latitude','longitude','categories'],converters={'categories':string_to_array})
    df_business=areamanager.pois_in_area(city_area,df_business)
    df_business['categories']=df_business.categories.apply(category_filter)
    areamanager.poi_set_subarea(city_area,df_business,0.5)
    return df_business

In [10]:
city_pois("lasvegas").to_csv("../data/poi/lasvegas.csv",index=False)

Area selected: Las Vegas


In [11]:
city_pois("phoenix").to_csv("../data/poi/phoenix.csv",index=False)

Area selected: Phoenix


In [12]:
city_pois("charlotte").to_csv("../data/poi/charlotte.csv",index=False)

Area selected: Charlotte


In [13]:
city_pois("madison").to_csv("../data/poi/madison.csv",index=False)

Area selected: Madison


Users json to csv with only user_id and friends

In [14]:
fuser=open("../data/user.json")
all_data = list()
df = pd.DataFrame(columns=['user_id', 'friends'])
df.to_csv('../data/user.csv',index=False)

for i, line in enumerate(fuser):  
    # json to dict
    obj_json = json.loads(line)
    # add to the data collection
    all_data.append([obj_json['user_id'],
                     obj_json['friends'].split(', ')])
    
    if i % 100000==0:
        df = pd.DataFrame(all_data, columns=['user_id', 'friends'])
        df.to_csv('../data/user.csv',header=False,index=False,mode='a')
        df=None
        all_data = list()
# create the DataFrame

df = pd.DataFrame(all_data, columns=['user_id', 'friends'])
df.to_csv('../data/user.csv',header=False,index=False,mode='a')
df=None
all_data = list()

In [14]:
def string_to_array(string):
    if string == '':
        return list()
    return eval(string)
df_user=pd.read_csv("../data/user.csv",converters={'friends':string_to_array})

Export cities users with their friends that are in the city

In [52]:
def friend_filter_city(friends,df_city_users):
    return pd.merge(pd.DataFrame(friends,columns=['user_id']),df_city_users,on='user_id').loc[:,'user_id'].tolist()

for city_name in cities_name:
    df_city_users=pd.read_csv("../data/checkin/"+city_name+".csv",usecols=['user_id']).drop_duplicates().reset_index(drop=True)
    df_city_user=pd.merge(df_user,df_city_users,on='user_id')
    df_city_user['friends']=df_city_user.friends.apply(lambda friends: friend_filter_city(friends,df_city_users))
    df_city_user.to_csv("../data/user/"+city_name+".csv",index=False)

### Export cities with checkin from others cities

In [51]:
df_checkin=pd.read_csv("../data/checkin.csv")

df_checkin.count()

user_id        7908994
business_id    7908994
date           7908994
latitude       7908994
longitude      7908994
categories     7908994
dtype: int64

In [52]:
# df_diff_users_visited=df_checkin[['user_id','business_id']].drop_duplicates().reset_index(drop=True).\
# groupby('business_id').count().reset_index().rename(columns={"user_id":"diffusersvisited"})

# df_diff_users_visited=df_diff_users_visited[df_diff_users_visited['diffusersvisited']>=5]
# del df_diff_users_visited['diffusersvisited']
# df_checkin=pd.merge(df_checkin,df_diff_users_visited,on='business_id',how='inner')
# #df_checkin_city.count()

# df_checkin['Count']=df_checkin.groupby(['user_id'])['user_id'].transform('count')
# df_checkin=df_checkin[df_checkin['Count']>=20]
# del df_checkin['Count']

In [53]:
df_checkin=df_checkin.set_index("user_id")

In [55]:
for city_name in cities_name:
    ##users_id=pd.read_csv("../data/user/"+city_name+".csv")['user_id']
    ##df_checkin.loc[users_id].to_csv("../data/checkin/cities/"+city_name+".csv")
    #users_id=pd.read_csv("../data/user/"+city_name+".csv")['user_id']
    city_area=areamanager.delimiter_area(city_name)
    df_checkin_city=areamanager.pois_in_area(city_area,df_checkin.reset_index())
    df_diff_users_visited=df_checkin_city[['user_id','business_id']].drop_duplicates().reset_index(drop=True).\
    groupby('business_id').count().reset_index().rename(columns={"user_id":"diffusersvisited"})

    df_diff_users_visited=df_diff_users_visited[df_diff_users_visited['diffusersvisited']>=5]
    
    del df_diff_users_visited['diffusersvisited']
    df_checkin_city=pd.merge(df_checkin_city,df_diff_users_visited,on='business_id',how='inner')
    df_checkin_city['Count']=df_checkin_city.groupby(['user_id'])['user_id'].transform('count')
    df_checkin_city=df_checkin_city[df_checkin_city['Count']>=20]
    del df_checkin_city['Count']
    #df_checkin.loc[users_id].to_csv("../data/checkin/cities/"+city_name+".csv")
    df_checkin_city.to_csv("../data/checkin/cities/"+city_name+".csv",index=False)

Area selected: Madison


In [56]:
pd.read_csv("../data/checkin/cities/"+city_name+".csv")

,user_id,business_id,date,latitude,longitude,categories
0,MB7Z80PJxNyQM7keDpR2XQ,br1uQbc3YU5jCpYi46nIgQ,2018-04-04 00:09:03,43.069425,-89.389567,"['Food', 'Coffee & Tea', 'Cafes', 'Restaurants']"
1,4HwS2oCVQ1-f_kzHCv5X3A,br1uQbc3YU5jCpYi46nIgQ,2018-02-16 20:40:41,43.069425,-89.389567,"['Food', 'Coffee & Tea', 'Cafes', 'Restaurants']"
2,OLBH2Ds1wllrzpHlyojzJQ,br1uQbc3YU5jCpYi46nIgQ,2018-04-12 11:31:03,43.069425,-89.389567,"['Food', 'Coffee & Tea', 'Cafes', 'Restaurants']"
3,wd3xoNaDLib8dhQ7BxUl6g,br1uQbc3YU5jCpYi46nIgQ,2018-03-07 14:08:41,43.069425,-89.389567,"['Food', 'Coffee & Tea', 'Cafes', 'Restaurants']"
4,4HwS2oCVQ1-f_kzHCv5X3A,br1uQbc3YU5jCpYi46nIgQ,2018-02-16 20:48:49,43.069425,-89.389567,"['Food', 'Coffee & Tea', 'Cafes', 'Restaurants']"
5,qp-z1copnYQq7NPubYlNLw,uLnPW9K1yhNpgzvodCumbQ,2014-10-27 04:19:36,42.979252,-89.516184,"['Breweries', 'Restaurants', 'Food', 'Bars', '..."
6,e9bVxMuG-1NIJWqT_pDEmQ,uLnPW9K1yhNpgzvodCumbQ,2014-10-27 20:54:54,42.979252,-89.516184,"['Breweries', 'Restaurants', 'Food', 'Bars', '..."
7,SDsVaBzPJr8fdY4-m5bjdw,uLnPW9K1yhNpgzvodCumbQ,2013-07-14 19:51:15,42.979252,-89.516184,"['Breweries', 'Restaurants', 'Food', 'Bars', '..."
8,tBZ8UAQFooesK_kst774ug,uLnPW9K1yhNpgzvodCumbQ,2013-08-16 21:37:22,42.979252,-89.516184,"['Breweries', 'Restaurants', 'Food', 'Bars', '..."
9,tbycp0aRhRsiAVYUAVR_uQ,uLnPW9K1yhNpgzvodCumbQ,2014-10-28 19:50:38,42.979252,-89.516184,"['Breweries', 'Restaurants', 'Food', 'Bars', '..."
